In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%%javascript
$('.nbp-app-bar').toggle()

<IPython.core.display.Javascript object>

## Check out the 5 datasets

In [3]:
path = '../../kaggle_data/input/'
spray = pd.read_csv(path+'spray.csv')

In [63]:
spray.tail(5)

,Date,Time,Latitude,Longitude
14830,2013-09-05,8:34:11 PM,42.006587,-87.812355
14831,2013-09-05,8:35:01 PM,42.006192,-87.816015
14832,2013-09-05,8:35:21 PM,42.006022,-87.817392
14833,2013-09-05,8:35:31 PM,42.005453,-87.817423
14834,2013-09-05,8:35:41 PM,42.004805,-87.817460


In [60]:
print(spray['Latitude'].describe())
print(traps['Latitude'].describe())


count    14835.000000
mean        41.904828
std          0.104381
min         41.713925
25%         41.785001
50%         41.940075
75%         41.980978
max         42.395983
Name: Latitude, dtype: float64
count    10506.000000
mean        41.841139
std          0.112742
min         41.644612
25%         41.732984
50%         41.846283
75%         41.954690
max         42.017430
Name: Latitude, dtype: float64


In [59]:
spray['Longitude'].describe()

count    14835.000000
mean       -87.736690
std          0.067292
min        -88.096468
25%        -87.794225
50%        -87.727853
75%        -87.694108
max        -87.586727
Name: Longitude, dtype: float64

In [55]:
keepme=spray[spray['Date']=='2011-08-29']

In [ ]:
traps[traps['Latitude']<42.395983]

In [56]:
keepme.head(20)
keepme['Latitude'].describe()

count    95.000000
mean     42.392331
std       0.001581
min      42.389460
25%      42.391232
50%      42.392428
75%      42.393198
max      42.395983
Name: Latitude, dtype: float64

In [31]:
# Here's how to calculate distance in Python.
from geopy.distance import vincenty
newport_ri = (41.49008, -71.312796)
cleveland_oh = (41.499498, -81.695391)
print(vincenty(newport_ri, cleveland_oh).miles)

538.3904451566326


# Wednesday Lunch Session

In [5]:
spray.shape

(14835, 4)

In [35]:
traps=pd.read_csv(path+'train.csv')
traps.head(2)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0


In [39]:
# Convert lat-long into a single variable
traps['lat_long'] = list(zip(traps['Latitude'], traps['Longitude']))
traps.head(3)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,lat_long
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,"(41.95469, -87.800991)"
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,"(41.95469, -87.800991)"
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0,"(41.994991, -87.769279)"


In [62]:
# Calculate the distance manually
our_spot=(42.391623, -88.089163)
their_spot=(41.95469, -87.800991)
vincenty(their_spot, our_spot).miles

33.59075344958164

In [65]:
# Calculate the distance from the spray spot
our_spot=(42.004805, -87.817460)
traps['distance_2_sprayspot']=traps['lat_long'].apply(lambda row: vincenty(row, our_spot).miles)
traps.head(2)


,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,lat_long,distance_2_sprayspot,flag_sprayspot
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,"(41.95469, -87.800991)",3.561274,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,"(41.95469, -87.800991)",3.561274,0


In [66]:
# Describe that new variable
traps['distance_2_sprayspot'].describe()

count    10506.000000
mean        13.654023
std          7.951512
min          0.560297
25%          6.172817
50%         13.036921
75%         21.097714
max         27.861349
Name: distance_2_sprayspot, dtype: float64

In [70]:
# Create a flag if it is within 1 mile of the spray spot
traps['flag_sprayspot']=traps['distance_2_sprayspot'].apply(lambda row: row<1).astype(int)
print(len(traps[traps['distance_2_sprayspot']<1]))
traps[traps['distance_2_sprayspot']<1].head(2)

116


,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,lat_long,distance_2_sprayspot,flag_sprayspot
52,2007-06-05,"7200 North Oketo Avenue, Chicago, IL 60631, USA",CULEX RESTUANS,72,N OKETO AVE,T016,"7200 N OKETO AVE, Chicago, IL",42.011601,-87.811506,8,1,0,"(42.011601, -87.811506)",0.560297,1
110,2007-06-26,"7200 North Oketo Avenue, Chicago, IL 60631, USA",CULEX PIPIENS/RESTUANS,72,N OKETO AVE,T016,"7200 N OKETO AVE, Chicago, IL",42.011601,-87.811506,8,2,0,"(42.011601, -87.811506)",0.560297,1


In [74]:
# Let's only keep those ones
keepme_test=traps[traps['flag_sprayspot']==1]
keepme_test.shape

(116, 15)

In [75]:
keepme_test['NumMosquitos'].describe()

count    116.000000
mean      13.836207
std       15.615457
min        1.000000
25%        2.000000
50%        8.000000
75%       18.000000
max       50.000000
Name: NumMosquitos, dtype: float64

In [86]:
# Restrict to 2013
keepme_test['Year']=keepme_test['Date'].str.split('-').str[0].astype(int)
keepme_test=keepme_test[keepme_test['Year']==2013]
keepme_test.head(2)

/Users/austinlasseter/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,lat_long,distance_2_sprayspot,flag_sprayspot,DayRunner,Year
8188,2013-06-07,"7100 North Harlem Avenue, Chicago, IL 60631, USA",CULEX PIPIENS/RESTUANS,71,N HARLEM AVE,T233,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,1,0,"(42.009876, -87.807277)",0.630276,1,158,2013
8308,2013-06-14,"7100 North Harlem Avenue, Chicago, IL 60631, USA",CULEX PIPIENS/RESTUANS,71,N HARLEM AVE,T233,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,17,0,"(42.009876, -87.807277)",0.630276,1,165,2013


In [78]:
len(keepme_test['Date']<'2013-09-05')

116

In [88]:
# Let's create a running count of days within the year:
keepme_test['DayRunner']=pd.to_datetime(keepme_test['Date']).dt.dayofyear
keepme_test.head(2)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,lat_long,distance_2_sprayspot,flag_sprayspot,DayRunner,Year
8188,2013-06-07,"7100 North Harlem Avenue, Chicago, IL 60631, USA",CULEX PIPIENS/RESTUANS,71,N HARLEM AVE,T233,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,1,0,"(42.009876, -87.807277)",0.630276,1,158,2013
8308,2013-06-14,"7100 North Harlem Avenue, Chicago, IL 60631, USA",CULEX PIPIENS/RESTUANS,71,N HARLEM AVE,T233,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,17,0,"(42.009876, -87.807277)",0.630276,1,165,2013


In [93]:
# What about that specific date?
day='2013-09-05'
day_num=248

In [101]:
keepme_test[(keepme_test['DayRunner']<247) | (keepme_test['DayRunner']>242)]  ['NumMosquitos'].mean()

19.558823529411764

In [95]:
keepme_test[keepme_test[('DayRunner']>=248)]['NumMosquitos'].mean()

16.833333333333332

In [6]:
# How many spray dates are there?
spray['Date'].nunique()

10

In [7]:
# Spraying only occurred in 10 days over 2 years (mostly in 2013)
spray_dates=list(spray['Date'].unique())
spray_dates

['2011-08-29',
 '2011-09-07',
 '2013-07-17',
 '2013-07-25',
 '2013-08-08',
 '2013-08-15',
 '2013-08-16',
 '2013-08-22',
 '2013-08-29',
 '2013-09-05']

In [14]:
spray.groupby('Date').mean()

,Latitude,Longitude
Date,,
2011-08-29,42.392331,-88.093370
2011-09-07,41.983344,-87.805927
2013-07-17,41.798570,-87.681137
2013-07-25,41.954535,-87.722879
2013-08-08,41.931466,-87.697035
2013-08-15,41.926007,-87.783171
2013-08-16,41.957809,-87.780646
2013-08-22,41.814052,-87.666029
2013-08-29,41.860128,-87.703237


In [13]:
x.shape

(10, 2)

In [8]:
# Let's create a running count of days within the year:
raw['DayRunner']=pd.to_datetime(raw['Date']).dt.dayofyear
print(raw['DayRunner'].describe())a
raw[['DayRunner', 'MonthDay', 'Month', 'Day']].head()

NameError: name 'raw' is not defined

In [ ]:
spray_days=list(raw['DayRunner'].unique().astype(int))
spray_days

In [ ]:
spray_downs=[x + 15 for x in spray_days]
spray_downs

In [ ]:
spray_windows=zip(spray_days, spray_downs)
list(spray_windows)

In [ ]:
# Any day-runner that falls within these window should be considered to have the spray effect.
# Note that the first two are 2011, the rest are 2013.
# Of course, they aren't all in the same location.

In [ ]:
# This info has been incorporated into the trap_readings dataset.